# Install of Julia
only needed if not already installed

In [ ]:
%%shell
set -e

VERSION="1.5.3"
# if the VERSION is altered the metadata of the notebook itself 
# (open the .ipynb file with a text editor [line 20]) has to be altered as well
# otherwise highlighting and autocompletion won't work

#-------------------------------------------------------------------------------

if [ -z `which julia` ]; then
  echo "Julia not found installing ..."
  URL="https://julialang-s3.julialang.org/bin/linux/x64/$(cut -d '.' -f -2 <<< "$VERSION")/julia-$VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  julia -e 'using Pkg; pkg"add IJulia; precompile;"'

  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict("JULIA_NUM_THREADS"=>"'"8"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo "Finished"
fi

Julia not found installing ...
2021-03-29 02:16:02 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.5/julia-1.5.3-linux-x86_64.tar.gz [105260711/105260711] -> "/tmp/julia.tar.gz" [1]
 Installing known registries into `~/.julia`
######################################################################## 100.0%
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed Conda ─────────── v1.5.1
  Installed VersionParsing ── v1.2.0
  Installed ZeroMQ_jll ────── v4.3.2+6
  Installed libsodium_jll ─── v1.0.18+1
  Installed SoftGlobalScope ─ v1.1.0
  Installed MbedTLS ───────── v1.0.3
  Installed Parsers ───────── v1.1.0
  Installed ZMQ ───────────── v1.2.1
  Installed IJulia ────────── v1.23.2
  Installed JSON ──────────── v0.21.1
  Installed JLLWrappers ───── v1.2.0
  Installed Artifacts ─────── v1.3.0
  Installed MbedTLS_jll ───── v2.16.8+1
######################################################################## 100.0%
############

# Installing needed packages
For SciBert Cuda Flux and Transformers get installed

as well as loading them

and setting some env variables

In [36]:
using Pkg
Pkg.add("CUDA")
Pkg.add("Flux")
Pkg.add("Transformers")
Pkg.add("DataDeps")
Pkg.add("DataFrames")
Pkg.add("JSON3")
Pkg.add("Metrics")


using Printf
using DataFrames
using JSON3
using Flux
using CUDA
using Metrics
using Transformers
using Transformers.Basic
using Transformers.Pretrain
using DataDeps
using Flux
using Flux: onehotbatch
using Flux: gradient,onehot
using Flux.Optimise: update!


enable_gpu(true)
ENV["DATADEPS_ALWAYS_ACCEPT"] = true

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/

true

defining the needed data

In [5]:
register(DataDep("SciBert",
    """
    Dataset: SciBERT
    Website: https://github.com/allenai/scibert
    """
    ,
    "https://codeload.github.com/allenai/scibert/zip/master",
    post_fetch_method = file ->(unpack(file),
                        #replace(file, ".zip" => "") ,
                        print(file,"\n"),
                        print("$(SubString(file,1,findlast(==('.'),file).-1))/data/\n"),
                        print(SubString.(file,1,findlast.(==('/'),file)), "\n"),
                        mv("$(SubString.(file,1,findlast.(==('.'),file).-1))/data/","$(SubString.(file,1,findlast.(==('/'),file)))/data" ))
))

DataDep{Nothing,String,typeof(DataDeps.fetch_default),var"#1#2"}("SciBert", "https://codeload.github.com/allenai/scibert/zip/master", nothing, DataDeps.fetch_default, var"#1#2"(), "Dataset: SciBERT\nWebsite: https://github.com/allenai/scibert\n")

In [6]:
df = JSON3.read.(eachline(datadep"SciBert/data/text_classification/chemprot/train.txt")) |> DataFrame
test = JSON3.read.(eachline(datadep"SciBert/data/text_classification/chemprot/test.txt")) |> DataFrame
@show

┌ Warning: Checksum not provided, add to the Datadep Registration the following hash line
│   hash = a3aac93a2b4f34c482c90c96a132f396cf473379d94c856fe3e84b144fda49d4
└ @ DataDeps /root/.julia/packages/DataDeps/ooWXe/src/verification.jl:44



7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
1 file, 28538447 bytes (28 MiB)

Extracting archive: /root/.julia/datadeps/SciBert/scibert-master.zip
--
Path = /root/.julia/datadeps/SciBert/scibert-master.zip
Type = zip
Physical Size = 28538447
Comment = 06793f77d7278898159ed50da30d173cdc8fdea9

Everything is Ok

Folders: 25
Files: 93
Size:       123880826
Compressed: 28538447
/root/.julia/datadeps/SciBert/scibert-master.zip
/root/.julia/datadeps/SciBert/scibert-master/data/
/root/.julia/datadeps/SciBert/


# Required functions


1.   loss function
2.   train function
3.   tokenize function
4.   score function (for calculating F1 score)

In [34]:
#define the loss
function loss(data, label, mask=nothing)
    e = model.embed(data)
    t = model.transformers(e, mask)
    l = Basic.logcrossentropy(
        label,
        model.classifier.clf(
            model.classifier.pooler(
                t[:,1,:]
            )
        )
    )
    return l
end

loss (generic function with 2 methods)

In [12]:
markline(s1) = ["[CLS]"; s1; "[SEP]"; s2; "[SEP]"]

function tokenize(sentence,label)
    attention_start = findfirst("<<",sentence)[1]
    attention_end  = findfirst(">>", sentence)[1]

    token_start = findfirst("[[", sentence)[1]
    token_end = findfirst("]]", sentence)[1]

    if attention_start < token_start
      b = split(sentence, r"<<|>>|\[\[|\]\]")

      output = ["[CLS]";b[1]|>tokenizer |> wordpiece; "[<<]" ;
        b[2]|>tokenizer|> wordpiece; "[>>]";
        b[3]|>tokenizer|> wordpiece; "[[[]";
        b[4]|>tokenizer|> wordpiece; "[]]]";
        b[5]|>tokenizer|> wordpiece ;"[SEP]"]
    else
      b = split(sentence, r"<<|>>|\[\[|\]\]")
      output = ["[CLS]";b[1]|>tokenizer|> wordpiece; "[[[]";
        b[2]|>tokenizer|> wordpiece; "[]]]";
        b[3]|>tokenizer |> wordpiece; "[<<]" ;
        b[4]|>tokenizer|> wordpiece; "[>>]";
        b[5]|>tokenizer|> wordpiece ;"[SEP]"]
    end


    tok = vocab(output)
    segment = fill!(similar(tok), 1)
    label = onehot(label, labels)
    mask = getmask([output])

    return (tok=tok, segment=segment), label, mask
end

tokenize (generic function with 1 method)

In [31]:
function train!()
  @info "start training"
  for i ∈ 1:nrow(df)
    sentence = df[!,"text"][i]
    label = df[!,"label"][i]

    #moving the needed data to the gpu
    data, label, mask = todevice(
      tokenize(sentence,label)
    )


    l = loss(data,label,mask)
    @show l
    #grad = gradient(()->l, ps)
    
    # new version of the gradient calculation
    grad = gradient(ps) do
              training_loss = loss(data,label,mask)
              training_loss
            end


    # update the model weights
    update!(opt, ps, grad)
  end
end

train! (generic function with 1 method)

In [43]:
function compute_F1(sentence,label, avg_type = "micro")
    Flux.testmode!(model)
    size = length(labels)

    y_pred = Array{Float64}(undef,size, 0)
    y_true = Array{Float64}(undef,size, 0)

    for i ∈ 1:length(sentence)
      flush(stdout)
      data, label_ignore, mask = todevice( #move data to gpu
                            tokenize(sentence[i],label[i])
                          )
      e =           model.embed(data)
      t =           model.transformers(e, mask)
      prediction =  model.classifier.clf(
                      model.classifier.pooler(
                        t[:,1,:]
                      )
                    )

      buffer_pred = Array(prediction)
      y_pred = hcat(y_pred,buffer_pred)

      pos = findall(x->x==label[i], labels)

      buffer_true = Metrics.onehot_encode(pos[1]-1, 0:size-1)
      y_true = hcat(y_true,buffer_true)
    end

    Flux.testmode!(model, false)

    return f_beta_score(y_pred, y_true; avg_type)
end

compute_F1 (generic function with 2 methods)

# Examples

###loading the test and trainings data of the chemprot corpus

In [ ]:
df = JSON3.read.(eachline(datadep"SciBert/data/text_classification/chemprot/train.txt")) |> DataFrame

#df.text = Vector{String}.(df.text)
#df.label = Vector{String}.(df.label)

@printf "printing column names of the train data:\n"
@show names(df)

@printf "\n\nfirst five labels of the training data:\n"
@show df[!,"label"][1:5]

@printf "\n\nfirst text of the training data:\n"
@show df[!,"text"][1]
df[!,"text"][1] |> scibert_scivocab_uncased_tokenizer |> scibert_scivocab_uncased_wordpiece

printing column names of the train data:
names(df) = ["text", "label", "metadata"]


first five labels of the training data:
(df[!, "label"])[1:5] = ["INHIBITOR", "INHIBITOR", "INHIBITOR", "INHIBITOR", "INHIBITOR"]


first text of the training data:
(df[!, "text"])[1] = "<< Epidermal growth factor receptor >> inhibitors currently under investigation include the small molecules [[ gefitinib ]] (Iressa, ZD1839) and erlotinib (Tarceva, OSI-774), as well as monoclonal antibodies such as cetuximab (IMC-225, Erbitux)."


67-element Array{String,1}:
 "<"
 "<"
 "epidermal"
 "growth"
 "factor"
 "receptor"
 ">"
 ">"
 "inhibitors"
 "currently"
 "under"
 "investigation"
 "include"
 ⋮
 "##uximab"
 "("
 "im"
 "##c"
 "-"
 "225"
 ","
 "erb"
 "##it"
 "##ux"
 ")"
 "."

In [37]:
labels = unique(df[!,"label"])
labels

13-element Array{String,1}:
 "INHIBITOR"
 "ANTAGONIST"
 "AGONIST"
 "DOWNREGULATOR"
 "PRODUCT-OF"
 "SUBSTRATE"
 "INDIRECT-UPREGULATOR"
 "UPREGULATOR"
 "INDIRECT-DOWNREGULATOR"
 "ACTIVATOR"
 "AGONIST-ACTIVATOR"
 "AGONIST-INHIBITOR"
 "SUBSTRATE_PRODUCT-OF"

In [42]:
@show test_label = test[!,"label"][1]

pos = findall(x->x==test_label, labels)
buffer_true = Metrics.onehot_encode(pos[1], 0:length(labels))

test_label = (test[!, "label"])[1] = "ANTAGONIST"


14×1 Array{Float64,2}:
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

# Rebuilding of SciBert for CLS task

defining the model, its parameters and the optimiser with the learning rate


In [10]:
model, wordpiece, tokenizer = pretrain"Bert-scibert_scivocab_uncased"

vocab = Vocabulary(wordpiece)
labels = unique(df[!,"label"])

show(model)
show(model.classifier.pooler)

#defining clf layer with dropout
clf = Chain(
    Dropout(0.1),
    Dense(size(scibert_scivocab_uncased_model.classifier.pooler.W ,1), length(labels)),
)

#redefining the scibert model
model =gpu(
    Basic.set_classifier(model,
                   (
                       pooler = model.classifier.pooler,
                       clf = clf
                   )
                  )
)
show(model)

┌ Info: loading pretrain bert model: scibert_scivocab_uncased.tfbson 
└ @ Transformers.BidirectionalEncoder /root/.julia/packages/Transformers/lFBL6/src/bert/load_pretrain.jl:8


TransformerModel{Bert{Stack{NTuple{12,Transformer{Transformers.Basic.MultiheadAttention{Dense{typeof(identity),Array{Float32,2},Array{Float32,1}},Dense{typeof(identity),Array{Float32,2},Array{Float32,1}},Dense{typeof(identity),Array{Float32,2},Array{Float32,1}},Dense{typeof(identity),Array{Float32,2},Array{Float32,1}},Dropout{Float64,Colon}},LayerNorm{typeof(identity),Flux.Diagonal{Array{Float32,1}},Float32,1},Transformers.Basic.PwFFN{Dense{typeof(gelu),Array{Float32,2},Array{Float32,1}},Dense{typeof(identity),Array{Float32,2},Array{Float32,1}}},LayerNorm{typeof(identity),Flux.Diagonal{Array{Float32,1}},Float32,1},Dropout{Float64,Colon}}},Symbol("((x, m) => x':(x, m)) => 12")},Dropout{Float64,Colon}}}(
  embed = CompositeEmbedding(tok = Embed(768), segment = Embed(768), pe = PositionEmbedding(768, max_len=512), postprocessor = Positionwise(LayerNorm((768,)), Dropout(0.1))),
  transformers = Bert(layers=12, head=12, head_size=64, pwffn_size=3072, size=768),
  classifier = 
    (
      p

defining the optimiser and the loss function


In [ ]:
ps = params(scibert_scivocab_uncased_model)
opt = ADAM(1e-5)

Test area

In [ ]:
text1 = "Peter Piper picked a peck of pickled peppers" |> scibert_scivocab_uncased_tokenizer |> scibert_scivocab_uncased_wordpiece
text2 = "Fuzzy Wuzzy was a bear" |> scibert_scivocab_uncased_tokenizer |> scibert_scivocab_uncased_wordpiece
google = "Google" |> scibert_scivocab_uncased_tokenizer |> scibert_scivocab_uncased_wordpiece

vocab = Vocabulary(scibert_scivocab_uncased_wordpiece)
labels = ("test","no_test")

text = ["[CLS]"; text1; "[SEP]"; text2; "[SEP]"]
text = ["[CLS]"; text1; "[<<]"; google; "[>>]"; "[[[]"; google; "[]]]"; text2; "[SEP]"]

token_indices = vocab(text)
mask = getmask([text])
segment_indices = [fill(1, length(text1)+2); fill(2, length(text2)+1)]
#segment_indices = [fill(1, length(text1)+2)]


tok = vocab(text)
segment = fill!(similar(tok), 1)

@show text
@show tok
@show token_indices

@show segment
@show segment_indices

sample = (tok = tok, segment = segment)
@show typeof(labels[1])
label = onehotbatch(["test"], labels)
#label = onehotbatch(["test"], labels)

text = ["[CLS]", "peter", "piper", "picked", "a", "pec", "##k", "of", "pick", "##led", "pep", "##pers", "[<<]", "google", "[>>]", "[[[]", "google", "[]]]", "fuzzy", "wu", "##zz", "##y", "was", "a", "bear", "[SEP]"]
tok = [103, 13053, 24461, 21554, 107, 11386, 30136, 132, 8767, 811, 11250, 11555, 102, 13388, 102, 102, 13388, 102, 4943, 7868, 10208, 30127, 242, 107, 12073, 104]
token_indices = [103, 13053, 24461, 21554, 107, 11386, 30136, 132, 8767, 811, 11250, 11555, 102, 13388, 102, 102, 13388, 102, 4943, 7868, 10208, 30127, 242, 107, 12073, 104]
segment = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
segment_indices = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2]
typeof(labels[1]) = String


2×1 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1
 0

In [ ]:
sample, label, mask = todevice(sample,label,mask)

((tok = [103, 13053, 24461, 21554, 107, 11386, 30136, 132, 8767, 811  …  13388, 102, 4943, 7868, 10208, 30127, 242, 107, 12073, 104], segment = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), Bool[1; 0], Float32[1.0 1.0 … 1.0 1.0])

In [44]:
sentence = test[!,"text"]
label = test[!,"label"]

new = compute_F1(sentence,label)

LoadError: ignored

In [33]:
train!()

l = -0.96681714f0
l = 

┌ Info: start training
└ @ Main In[31]:2


-0.96671283f0
l = -0.9671825f0
l = -0.96695006f0
l = -0.96727383f0
l = -0.96707296f0
l = -0.9683845f0
l = -0.968818f0
l = -0.96892774f0
l = -0.95700514f0
l = -0.9575579f0
l = -0.9589938f0
l = -0.9596431f0
l = -0.96037817f0
l = -0.9580173f0
l = -0.95889914f0
l = -0.9598967f0
l = 0.34210283f0
l = 0.33524504f0
l = -0.9446028f0
l = -0.94264394f0
l = -0.4604613f0
l = 0.30619475f0
l = 0.3032583f0
l = 0.32196876f0
l = 0.31922302f0
l = 1.7618927f0
l = -0.8176381f0
l = -0.2317341f0
l = -0.94821227f0
l = -0.94778234f0
l = -0.9481677f0
l = -0.94797075f0
l = -0.9481932f0
l = -0.94797117f0
l = -0.31923574f0
l = -0.04273933f0
l = -0.042414755f0
l = -0.96529347f0
l = -0.9641266f0
l = -0.9480587f0
l = -0.94804f0
l = -0.95895433f0
l = -0.95898706f0
l = -0.95804644f0
l = -0.958091f0
l = -0.958099f0
l = -0.9587119f0
l = -0.27564624f0
l = -0.4118296f0
l = 0.33806974f0
l = -0.18791796f0
l = -0.18813017f0
l = -0.18818101f0
l = -0.18796556f0
l = -0.18807036f0
l = -0.1881578f0
l = -0.18836989f0
l = -0.2274219

LoadError: ignored

In [ ]:
text1 = "Peter Piper picked a peck of pickled peppers" |> scibert_scivocab_uncased_tokenizer |> scibert_scivocab_uncased_wordpiece
text2 = "Fuzzy Wuzzy was a bear" |> scibert_scivocab_uncased_tokenizer |> scibert_scivocab_uncased_wordpiece
vocab = Vocabulary(scibert_scivocab_uncased_wordpiece)

text = ["[CLS]"; text1; "[SEP]"; text2; "[SEP]"]

token_indices = vocab(text)
segment_indices = [fill(1, length(text1)+2); fill(2, length(text2)+1)]
sample = (tok = token_indices, segment = segment_indices)

bert_embedding = sample |> scibert_scivocab_uncased_model.embed
feature_tensors = bert_embedding |> scibert_scivocab_uncased_model.transformers

scibert_scivocab_uncased_model.classifier.clf(scibert_scivocab_uncased_model.classifier.pooler(feature_tensors[:,1,:]))

2×1 CuArray{Float32,2}:
 0.24135572
 0.551603

# Loading diffrent SciBert models

Loading of the uncased SciBert model with scivocab

In [ ]:
using Flux
using CUDA
using Transformers
using Transformers.Basic
using Transformers.Pretrain

ENV["DATADEPS_ALWAYS_ACCEPT"] = true


scibert_scivocab_uncased_model, scibert_scivocab_uncased_wordpiece, scibert_scivocab_uncased_tokenizer = pretrain"Bert-scibert_scivocab_uncased"
scibert_basevocab_cased_model, scibert_basevocab_cased_wordpiece, scibert_basevocab_cased_tokenizer = pretrain"Bert-scibert_basevocab_cased"
scibert_basevocab_uncased_model, scibert_basevocab_uncased_wordpiece, scibert_basevocab_uncased_tokenizer = pretrain"Bert-scibert_basevocab_uncased"
scibert_scivocab_cased_model, scibert_scivocab_cased_wordpiece, scibert_scivocab_cased_tokenizer = pretrain"Bert-scibert_scivocab_cased"


#show(bert_model)

┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1278


######################################################################### 100.0%


######################################################################### 100.0%


######################################################################### 100.0%


┌ Info: Precompiling Transformers [21ca0261-441d-5938-ace7-c90938fde4d4]
└ @ Base loading.jl:1278
┌ Info: Downloading
│   source = https://docs.google.com/uc?export=download&id=1L0woI2DeNES5OCzgLNBOCRrj5ikOzN7c
│   dest = /root/.julia/datadeps/BERT-scibert_scivocab_uncased/scibert_scivocab_uncased.tfbson
│   progress = 1.0
│   time_taken = 22.04 s
│   time_remaining = 0.0 s
│   average_speed = 19.162 MiB/s
│   downloaded = 422.264 MiB
│   remaining = 0 bytes
│   total = 422.264 MiB
└ @ Transformers.Datasets /root/.julia/packages/Transformers/ko7g9/src/datasets/download_utils.jl:116
┌ Info: loading pretrain bert model: scibert_scivocab_uncased.tfbson 
└ @ Transformers.BidirectionalEncoder /root/.julia/packages/Transformers/ko7g9/src/bert/load_pretrain.jl:8
┌ Info: Downloading
│   source = https://docs.google.com/uc?export=download&id=1Htg-qTj03YRQqBgbHxz8KOULBYQGQieP
│   dest = /root/.julia/datadeps/BERT-scibert_basevocab_cased/scibert_basevocab_cased.tfbson
│   progress = 1.0
│   time_

LoadError: ignored

In [ ]:
text1 = "Peter Piper picked a peck of pickled peppers" |> scibert_scivocab_uncased_tokenizer |> scibert_scivocab_uncased_wordpiece
text2 = "Fuzzy Wuzzy was a bear" |> scibert_scivocab_uncased_tokenizer |> scibert_scivocab_uncased_wordpiece
vocab = Vocabulary(scibert_scivocab_uncased_wordpiece)

text = ["[CLS]"; text1; "[SEP]"; text2; "[SEP]"]

token_indices = vocab(text)
segment_indices = [fill(1, length(text1)+2); fill(2, length(text2)+1)]
sample = (tok = token_indices, segment = segment_indices)

bert_embedding = sample |> scibert_scivocab_uncased_model.embed
feature_tensors = bert_embedding |> scibert_scivocab_uncased_model.transformers

768×21 Array{Float32,2}:
 -0.941097    0.312237     0.0372119  …   0.320879     0.365936   -0.672109
  0.267526    0.392251    -0.230857       0.0250972    0.334235   -0.254514
  0.733429   -0.312628    -0.0411342     -0.412593    -0.309975    0.854437
 -0.444379    0.299888    -0.103102      -0.560207    -0.0803011   0.518665
 -0.446466   -0.421271    -1.18789       -0.649227    -0.594699   -0.548666
 -0.0353962   0.109076     0.317547   …   0.77719      0.0578743   0.182419
 -0.187649    0.572562     0.239817       0.729143    -0.0142986   0.396244
 -1.57516    -0.515017    -0.018913      -1.59524     -0.739407   -1.22416
  0.437413    0.00692399   0.689993      -0.0514277    0.148664    1.10192
  0.0766271   0.407363     1.02306       -0.288707    -0.433671    0.943716
 -0.066848    0.102996     0.689254   …  -0.285827     0.128693   -0.381973
  0.59927     0.24867      0.642632       0.326496    -0.132859    0.139623
 -0.558318   -0.189067     0.44933        0.060011     0.198506  

In [ ]:
enable_gpu(true)
embed = Embed(512, length(vocab)) |> gpu
#define a position embedding layer metioned above
pe = PositionEmbedding(512) |> gpu

function embedding(x)
  we = embed(x, inv(sqrt(512))) 
  e = we .+ pe(we)
  return e
end

function encoder_forward(x)
  e = embedding(x)
  t1 = scibert_scivocab_uncased_model(e)
  return t1
end

linear = Positionwise(Dense(512, length(vocab)), logsoftmax) |> gpu

Positionwise(Dense(512, 31090), logsoftmax)

In [ ]:
preprocess(x) = [startsym, x..., endsym]

@show sample = preprocess.(sample_data())
@show encoded_sample = vocab(sample[1]) #use Vocabulary to encode the training data

LoadError: ignored

In [ ]:
using Transformers.Datasets
using Transformers.Datasets.GLUE
using Transformers.Basic
using Flux: onehotbatch

task = GLUE.QNLI()
labels = get_labels(task)
typeof(labels[1])

String

In [ ]:
enc = encoder_forward()

LoadError: ignored

In [ ]:
scibert_scivocab_uncased_model(vocab("Hund"))

LoadError: ignored

In [ ]:
const hidden_size = size(scibert_scivocab_uncased_model.classifier.pooler.W ,1)

const clf = gpu(Chain(
    Dropout(0.1),
    Dense(hidden_size, length(labels)),
    logsoftmax
))

const bert_model = gpu(
    set_classifier(_bert_model,
                   (
                       pooler = _bert_model.classifier.pooler,
                       clf = clf
                   )
                  )
)

LoadError: ignored

SyntaxError: ignored

In [ ]:
using Flux
using Flux: onehot, onecold
using Transformers
using Transformers.Basic

labels = collect(1:10)
startsym = 11
endsym = 12
unksym = 0
labels = [unksym, startsym, endsym, labels...]
vocab = Vocabulary(labels, unksym)

#function for generate training datas
sample_data() = (d = rand(1:10, 10); (d,d))
#function for adding start & end symbol
preprocess(x) = [startsym, x..., endsym]

@show sample = preprocess.(sample_data())
@show encoded_sample = vocab(sample[1]) #use Vocabulary to encode the training data


#define a Word embedding layer which turn word index to word vector
embed = Embed(512, length(vocab))
#define a position embedding layer metioned above
pe = PositionEmbedding(512)

#wrapper for get embedding
function embedding(x)
  we = embed(x, inv(sqrt(512))) 
  e = we .+ pe(we)
	return e
end

#define 2 layer of transformer
encode_t1 = Transformer(512, 8, 64, 2048)
encode_t2 = Transformer(512, 8, 64, 2048)

#define 2 layer of transformer decoder
decode_t1 = TransformerDecoder(512, 8, 64, 2048) 
decode_t2 = TransformerDecoder(512, 8, 64, 2048)

#define the layer to get the final output probabilities
linear = Positionwise(Dense(512, length(vocab)), logsoftmax)

function encoder_forward(x)
  e = embedding(x)
  t1 = encode_t1(e)
  t2 = encode_t2(t1)
  return t2
end

function decoder_forward(x, m)
  e = embedding(x)
  t1 = decode_t1(e, m)
  t2 = decode_t2(t1, m)
  p = linear(t2)
	return p
end

enc = encoder_forward(encoded_sample)
probs = decoder_forward(encoded_sample, enc)

function smooth(et)
    sm = fill!(similar(et, Float32), 1e-6/size(embed, 2))
    p = sm .* (1 .+ -et)
    label = p .+ et .* (1 - convert(Float32, 1e-6))
    label
end

#define loss function
function loss(x, y)
  label = onehot(vocab, y) #turn the index to one-hot encoding
  label = smooth(label) #perform label smoothing
  enc = encoder_forward(x)
	probs = decoder_forward(y, enc)
  l = logkldivergence(label[:, 2:end, :], probs[:, 1:end-1, :])
  return l
end

#collect all the parameters
ps = params(embed, pe, encode_t1, encode_t2, decode_t1, decode_t2, linear)
opt = ADAM(1e-4)

#function for created batched data
using Transformers.Datasets: batched

#flux function for update parameters
using Flux: gradient
using Flux.Optimise: update!

#define training loop
function train!()
  @info "start training"
  for i = 1:2000
    data = batched([sample_data() for i = 1:32]) #create 32 random sample and batched
		x, y = preprocess.(data[1]), preprocess.(data[2])
    x, y = vocab(x), vocab(y)#encode the data
    x, y = todevice(x, y) #move to gpu
    l = loss(x, y)
    grad = gradient(()->l, ps)
    if i % 8 == 0
    	println("loss = $l")
    end
    update!(opt, ps, grad)
  end
end


train!()


using Flux: onecold
function translate(x)
    ix = todevice(vocab(preprocess(x)))
    seq = [startsym]

    enc = encoder_forward(ix)

    len = length(ix)
    for i = 1:2len
        trg = todevice(vocab(seq))
        dec = decoder_forward(trg, enc)
        #move back to gpu due to argmax wrong result on CuArrays
        ntok = onecold(collect(dec), labels)
        push!(seq, ntok[end])
        ntok[end] == endsym && break
    end
  seq[2:end-1]
end

translate([5,5,6,6,1,2,3,4,7, 10])